In [1]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import ttk

from joblib import load

In [2]:
# Carga transforms. guardados
encoder_loaded = load('../4.2 models_prepro/encoder.joblib')
scaler_loaded = load('../4.2 models_prepro/scaler.joblib')
imputer_loaded = load('../4.2 models_prepro/imputer.joblib')
model_class = load('../5. Modelo_ML/model_classification.joblib')
model_reg = load('../5. Modelo_ML/model_regression.joblib')

# Ejemplo de datos nuevos
data = {
    'names': ['Madrid', 'Sevilla', 'Barcelona'],
    'comunity': ['Madrid', 'Andalucia', 'Cataluña'],
    'scores': [101, 75, 85],
    'dates': ['2024-07-04', '2024-07-04', '2024-07-04'],
    'co': [0, 0.6, 0.7],
    'humidity': [19, 55, 60],
    'no2': [6, 0.02, 0.03],
    'o3': [73, 0.04, 0.05],
    'p': [1015, 1014, 1015],
    'pm10': [101, 25, 30],
    'pm25': [87, 12, 15],
    'so2': [2, 0.003, 0.004],
    'temp': [36, 26, 27],
    'wind': [3, 6, 7],
    'Dia': ['2024-07-04', '2024-07-04', '2024-07-04'],
    'Hora': [8, 12, 16],
    'periodo': ['mañana', 'mediodia', 'tarde'],
    'calidad_aire': ['Dañina a la Salud de los Grupos Sensitivos', 'Buena', 'Moderada']
}

# Crear DataFrame
df = pd.DataFrame(data)

new_df = df
new_df['dates'] = pd.to_datetime(new_df['dates'], errors='coerce')
new_df['Dia'] = pd.to_datetime(new_df['Dia'])
new_df['scores'] = pd.to_numeric(new_df['scores'], errors='coerce')

# Definir cols categoricas y numericas
categorical_features = ['names', 'comunity', 'periodo', 'calidad_aire']
numeric_features = new_df.select_dtypes(include=['float64', 'int64']).columns.tolist()
numeric_features.remove('scores')

for col in categorical_features:
    new_df[col] = new_df[col].astype('category')

periodo_mapping = {'mañana': 0, 'mediodia': 1, 'tarde': 2, 'noche': 3, 'otro': 4}
calidad_aire_mapping = {'Muy Dañina a la Salud': 0, 'Dañina a la Salud': 1, 'Dañina a la Salud de los Grupos Sensitivos': 2, 'Moderada': 3, 'Buena': 4, '-': 5}
new_df['periodo_encoded'] = new_df['periodo'].map(periodo_mapping)
new_df['calidad_aire_encoded'] = new_df['calidad_aire'].map(calidad_aire_mapping)

df_enc = pd.DataFrame(encoder_loaded.transform(new_df[categorical_features]), columns=encoder_loaded.get_feature_names_out(categorical_features))
df_imp = pd.DataFrame(imputer_loaded.transform(new_df[numeric_features]), columns=numeric_features)
df_add = pd.concat([new_df['scores'], new_df['periodo_encoded'], new_df['calidad_aire_encoded']], axis=1)
df_pred = pd.concat([df_add, df_imp, df_enc], axis=1)
df_pred[numeric_features] = scaler_loaded.transform(df_pred[numeric_features])

next_class_encoded = model_class.predict(df_pred)
col_name = 'next_class_encoded'
col_position = 3
df_pred[col_name] = next_class_encoded
df_pred.insert(col_position, col_name, df_pred.pop(col_name))

next_score = model_reg.predict(df_pred)
df_pred['next_score'] = next_score

print(df_pred[['next_class_encoded', 'next_score']])

# Mapeo de la clase predicha a su descripción
class_mapping = {0: 'Muy Dañina a la Salud', 1: 'Dañina a la Salud', 2: 'Dañina a la Salud de los Grupos Sensitivos', 3: 'Moderada', 4: 'Buena', 5: '-'}

# Definir el orden de los periodos
periodo_mapping = {'mañana': 0, 'mediodia': 1, 'tarde': 2, 'noche': 3, 'otro': 4}
inverse_periodo_mapping = {v: k for k, v in periodo_mapping.items()}

# Función para obtener el siguiente periodo
def get_next_period(period):
    current_index = periodo_mapping[period]
    next_index = (current_index + 1) % 4  # Para hacer circular el ciclo
    return inverse_periodo_mapping[next_index]

# Agregar las nuevas columnas
df_pred['names'] = new_df['names']
df_pred['current_periodo'] = new_df['periodo']
df_pred['next_periodo'] = df_pred['current_periodo'].apply(get_next_period)

# Crear la ventana de Tkinter
root = tk.Tk()
root.title("Predicciones de Calidad de Aire")

# Crear el Treeview
tree = ttk.Treeview(root, columns=("names", "current_periodo", "next_periodo", "next_class", "next_score"), show='headings')
tree.heading("names", text="Names")
tree.heading("current_periodo", text="Current Periodo")
tree.heading("next_periodo", text="Next Periodo")
tree.heading("next_class", text="Next Class")
tree.heading("next_score", text="Next Score")
tree.grid(row=0, column=0, sticky='nsew')

# Funciones para obtener el color de fondo basado en la gravedad
def get_class_bg_color(next_class):
    if next_class == 'Muy Dañina a la Salud':
        return "red"
    elif next_class == 'Dañina a la Salud':
        return "orange"
    elif next_class == 'Dañina a la Salud de los Grupos Sensitivos':
        return "yellow"
    elif next_class == 'Moderada':
        return "light green"
    elif next_class == 'Buena':
        return "green"
    else:
        return "white"

def get_score_bg_color(score):
    if score <= 50:
        return "green"
    elif score <= 100:
        return "light green"
    elif score <= 150:
        return "yellow"
    elif score <= 200:
        return "orange"
    elif score <= 300:
        return "red"
    else:
        return "purple"


# Agregar predicciones al Treeview con colores de fondo independientes
for index, row in df_pred.iterrows():
    next_class = class_mapping[row['next_class_encoded']]
    next_score = row['next_score']
    
    class_bg_color = get_class_bg_color(next_class)
    score_bg_color = get_score_bg_color(next_score)
    
    tree.insert("", "end", values=(row['names'], row['current_periodo'], row['next_periodo'], next_class, next_score))
    
    tree.tag_configure(f'row_{index}_class', background=class_bg_color)
    tree.tag_configure(f'row_{index}_score', background=score_bg_color)
    tree.item(tree.get_children()[-1], tags=(f'row_{index}_class', f'row_{index}_score'))

# Ejecutar la ventana de Tkinter
root.mainloop()


   next_class_encoded  next_score
0                 3.0      66.560
1                 4.0      37.580
2                 4.0      41.185
